<a href="https://colab.research.google.com/github/1moi6/pyfuzzy-toolbox/blob/main/notebooks_colab/Aula_5/01_otimizacao_pso_funcao_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐝 Otimização de Sistemas Fuzzy com PSO

**Aula 5 - Minicurso de Sistemas de Inferência Fuzzy**

---

## 🎯 Objetivo

Neste notebook, vamos aplicar **PSO (Particle Swarm Optimization)** para otimizar parâmetros de sistemas fuzzy.

### Problema: Aproximar f(x) = -2x + 5

---

## 🐝 O que é PSO?

**PSO (Particle Swarm Optimization)** é inspirado no comportamento de **enxames**:

### Conceito:
- **Partículas**: Cada solução é uma "partícula" no espaço
- **Velocidade**: Partículas se movem baseadas em:
  - 🧠 Melhor posição **pessoal** (memória individual)
  - 🌍 Melhor posição **global** (conhecimento coletivo)
- **Convergência**: Enxame encontra melhores soluções colaborativamente

### Equações do PSO:

```
v[i] = w*v[i] + c1*r1*(pbest[i] - x[i]) + c2*r2*(gbest - x[i])
x[i] = x[i] + v[i]
```

Onde:
- `v[i]`: Velocidade da partícula i
- `x[i]`: Posição da partícula i  
- `w`: Inércia (0.7)
- `c1, c2`: Coeficientes cognitivo/social (1.5)
- `pbest`: Melhor posição pessoal
- `gbest`: Melhor posição global

---

## 📚 Referências
- Kennedy, J., & Eberhart, R. (1995). "Particle swarm optimization." *IEEE*

---

## 🔧 Instalação

In [ ]:
!pip install pyfuzzy-toolbox[ml] -q

print('✅ pyfuzzy-toolbox instalado com sucesso!')

## 📦 Importações

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import fuzzy_systems as fs
from fuzzy_systems.learning import PSO
from fuzzy_systems.learning import MamdaniNeurofuzzy

%matplotlib inline

# Configurações
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

print('✅ Bibliotecas importadas!')
print(f'   pyfuzzy-toolbox: {fs.__version__}')

## 📊 Passo 1: Gerar Dados - Função Linear

### f(x) = -2x + 5

In [ ]:
# Função alvo
def funcao_real(x):
    """f(x) = -2x + 5"""
    return -2 * x + 5

# Gerar dados
n_samples = 100
X_train = np.linspace(0, 10, n_samples).reshape(-1, 1)
y_train = funcao_real(X_train.flatten()) + np.random.normal(0, 0.2, n_samples)

X_test = np.linspace(0, 10, 200).reshape(-1, 1)
y_true = funcao_real(X_test.flatten())

# Visualizar
plt.figure(figsize=(10, 6))
plt.plot(X_test, y_true, 'k-', linewidth=3, label='f(x) = -2x + 5', alpha=0.7)
plt.scatter(X_train, y_train, color='red', s=30, alpha=0.5, label=f'Dados (n={n_samples})')
plt.xlabel('x', fontsize=13)
plt.ylabel('y', fontsize=13)
plt.title('Problema: Otimizar sistema fuzzy com PSO', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f'📊 Dados gerados:')
print(f'   • Função: f(x) = -2x + 5')
print(f'   • Amostras: {n_samples}')

## 🧩 Passo 2: Criar Sistema Fuzzy Mamdani

Vamos usar `MamdaniNeurofuzzy` que já está preparado para otimização com PSO!

In [ ]:
# Criar sistema Mamdani neuro-fuzzy
sistema = MamdaniNeurofuzzy(
    n_inputs=1,
    n_mfs=5,  # 5 MFs gaussianas
    input_ranges=[(0, 10)],
    output_range=(-20, 10)
)

print('✅ Sistema Fuzzy criado!')
print(f'\n📊 Configuração:')
print(f'   • Entrada: 1 variável, 5 MFs gaussianas')
print(f'   • Saída: 5 MFs triangulares')
print(f'   • Total de regras: 5')
print(f'   • Parâmetros otimizáveis: médias, sigmas, centroides')

## 🐝 Passo 3: Otimizar com PSO

O PSO vai otimizar:
- **Médias** das MFs gaussianas de entrada
- **Sigmas** das MFs gaussianas
- **Centroides** das MFs de saída

In [ ]:
# Configurar PSO
pso = PSO(
    n_particles=30,
    n_iterations=100,
    w=0.7,      # Inércia
    c1=1.5,     # Coeficiente cognitivo
    c2=1.5      # Coeficiente social
)

print('🐝 Iniciando otimização com PSO...')
print('   • 30 partículas')
print('   • 100 iterações máximas\n')

# Otimizar
best_params, best_fitness, history = pso.optimize_mamdani(
    sistema,
    X_train,
    y_train,
    verbose=True
)

print('\n' + '='*60)
print('✅ Otimização concluída!')
print('='*60)
print(f'\n📊 Resultados:')
print(f'   • Melhor RMSE: {best_fitness:.6f}')
print(f'   • Iterações: {len(history)}')

## 📊 Passo 4: Avaliar Sistema Otimizado

In [ ]:
# Predições
y_pred = sistema.predict(X_test)

# Métricas
rmse = np.sqrt(np.mean((y_pred - y_true) ** 2))
mae = np.mean(np.abs(y_pred - y_true))
r2 = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2))

print('='*70)
print('📊 MÉTRICAS DE DESEMPENHO - SISTEMA OTIMIZADO COM PSO')
print('='*70)
print(f'   • RMSE: {rmse:.6f}')
print(f'   • MAE:  {mae:.6f}')
print(f'   • R²:   {r2:.6f}')
print('='*70)

## 📈 Passo 5: Visualizar Resultados

In [ ]:
# Criar visualização
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Aproximação
axes[0, 0].plot(X_test, y_true, 'b-', linewidth=3, label='Real', alpha=0.7)
axes[0, 0].plot(X_test, y_pred, 'r--', linewidth=2.5, label='Fuzzy+PSO', alpha=0.8)
axes[0, 0].scatter(X_train, y_train, alpha=0.3, s=20, c='blue')
axes[0, 0].set_xlabel('x', fontsize=12)
axes[0, 0].set_ylabel('y', fontsize=12)
axes[0, 0].set_title(f'Aproximação\nRMSE = {rmse:.4f} | R² = {r2:.4f}', 
                     fontsize=13, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# 2. Erro
error = y_pred - y_true
axes[0, 1].plot(X_test, error, 'g-', linewidth=2.5)
axes[0, 1].axhline(y=0, color='k', linestyle='--')
axes[0, 1].fill_between(X_test.flatten(), 0, error, alpha=0.3, color='green')
axes[0, 1].set_xlabel('x', fontsize=12)
axes[0, 1].set_ylabel('Erro', fontsize=12)
axes[0, 1].set_title(f'Erro | MAE = {mae:.4f}', fontsize=13, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# 3. Convergência PSO
axes[1, 0].plot(history, linewidth=2.5, color='darkblue')
axes[1, 0].set_xlabel('Iteração', fontsize=12)
axes[1, 0].set_ylabel('Fitness (RMSE)', fontsize=12)
axes[1, 0].set_title('Convergência PSO', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Distribuição do Erro
axes[1, 1].hist(error, bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Erro', fontsize=12)
axes[1, 1].set_ylabel('Frequência', fontsize=12)
axes[1, 1].set_title('Distribuição do Erro', fontsize=13, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print('✅ Visualizações geradas!')

---

## 🎓 Conclusões

### ✅ O que aprendemos?

1. **PSO para Otimização Fuzzy**:
   - Otimiza **parâmetros das MFs** (não apenas regras)
   - **Enxame colabora** para encontrar melhores soluções
   - Evita **mínimos locais** via exploração coletiva

2. **Processo PSO**:
   - Cada **partícula** = Uma configuração do sistema fuzzy
   - **Velocidade** ajustada por memória pessoal + conhecimento global
   - **Convergência** natural do enxame

3. **Vantagens do PSO**:
   - ✅ Não precisa de **gradientes**
   - ✅ Fácil de implementar
   - ✅ Poucos hiperparâmetros
   - ✅ Funciona bem em espaços contínuos

---

### 🔬 PSO vs Outros Métodos

| Método | O que Otimiza | Vantagem |
|--------|---------------|----------|
| **Wang-Mendel** | Nada (gera regras) | Rápido, 1 passo |
| **ANFIS** | MFs via gradient | Preciso, teórico |
| **PSO** | MFs via enxame | Robusto, sem gradientes |
| **GA** | MFs via evolução | Exploração ampla |

---

### 🚀 Próximos Passos

1. **Testar outros algoritmos** (GA, DE)
2. **Funções não-lineares** (parábola, seno)
3. **Comparar PSO vs ANFIS**
4. **Otimizar hiperparâmetros** do PSO

---

### 📚 Recursos

- **PyPI**: https://pypi.org/project/pyfuzzy-toolbox/
- **GitHub**: https://github.com/1moi6/pyfuzzy-toolbox

### 📖 Referências

1. **Kennedy, J., & Eberhart, R. (1995)**. "Particle swarm optimization." *IEEE*

2. **Shi, Y., & Eberhart, R. (1998)**. "A modified particle swarm optimizer." *IEEE*

---

**🎉 Parabéns! Você aplicou PSO para otimizar sistemas fuzzy!**

*Notebook desenvolvido para o Minicurso de Sistemas de Inferência Fuzzy - Aula 5 - 2025*  
*Usando pyfuzzy-toolbox v1.0.0*